In [1]:
import pyspark
from pyspark.sql import SparkSession
import os

In [2]:
## DEFINE SENSITIVE VARIABLES
CATALOG_URI = "http://catalog:8181" 
WAREHOUSE = "s3a://datalakehouse" ## BUCKET TO WRITE DATA TOO
AWS_ACCESS_KEY = os.environ.get("AWS_ACCESS_KEY", "admin") ## AWS CREDENTIALS
AWS_SECRET_KEY = os.environ.get("AWS_SECRET_KEY", "password") ## AWS CREDENTIALS
AWS_S3_ENDPOINT= os.environ.get("AWS_S3_ENDPOINT", 'http://storage:9000') ## MINIO ENDPOINT


print(AWS_S3_ENDPOINT)
print(CATALOG_URI)
print(WAREHOUSE)

http://storage:9000
http://catalog:8181
s3a://datalakehouse


In [3]:
conf = (
    pyspark.SparkConf()
        .setAppName('app_name')
        .set('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.5.2,software.amazon.awssdk:bundle:2.28.28,software.amazon.awssdk:url-connection-client:2.28.28')
        .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions')
        .set('spark.sql.catalog.notebok_catalog', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.notebok_catalog.type', 'rest')
        .set('spark.sql.catalog.notebok_catalog.uri', CATALOG_URI)
        .set('spark.sql.catalog.notebok_catalog.authentication.type', 'NONE')
        .set('spark.sql.catalog.notebok_catalog.s3.endpoint', AWS_S3_ENDPOINT)
        .set('spark.sql.catalog.notebok_catalog.warehouse', WAREHOUSE)
        .set('spark.sql.catalog.notebok_catalog.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
        .set('spark.hadoop.fs.s3a.access.key', AWS_ACCESS_KEY)
        .set('spark.hadoop.fs.s3a.secret.key', AWS_SECRET_KEY)
)


## Start Spark Session
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark Running")

:: loading settings :: url = jar:file:/home/docker/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/docker/.ivy2/cache
The jars for the packages stored in: /home/docker/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.3_2.12 added as a dependency
software.amazon.awssdk#bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c1ff9a10-e379-40a2-b59d-bf38fc4b652f;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.3_2.12;1.5.2 in central
	found software.amazon.awssdk#bundle;2.28.28 in central
	found software.amazon.awssdk#url-connection-client;2.28.28 in central
	found software.amazon.awssdk#utils;2.28.28 in central
	found org.reactivestreams#reactive-streams;1.0.4 in central
	found software.amazon.awssdk#annotations;2.28.28 in central
	found org.slf4j#slf4j-api;1.7.36 in central
	found software.amazon.awssdk#http-client-spi;2.28.28 in central
	found software.amazon.awssdk#metrics-spi;2.28.28 in central
:: resolution report ::

24/11/08 23:14:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Spark Running


In [10]:
spark.sql("USE notebok_catalog").show()

++
||
++
++



In [11]:
spark.sql("SHOW SCHEMAS").show(truncate=False)

+---------+
|namespace|
+---------+
|lakehouse|
+---------+



In [13]:
spark.sql("SELECT * FROM lakehouse.lol_statistics").show(truncate=False)

+--------------+--------+-------------+---------+-----------+--------+------------+
|player_name   |rank    |main_champion|main_role|average_kda|win_rate|games_played|
+--------------+--------+-------------+---------+-----------+--------+------------+
|ArcaneMage    |Silver  |Veigar       |Mid      |3.2        |47      |70          |
|StormBreaker  |Gold    |Olaf         |Top      |2.8        |52      |84          |
|MysticArrow   |Gold    |Varus        |ADC      |3.1        |52      |88          |
|OceanReaper   |Diamond |Nami         |Support  |3.6        |59      |134         |
|SpellCaster   |Silver  |Lux          |Mid      |3.0        |47      |69          |
|SteelBlade    |Diamond |Fiora        |Top      |3.7        |58      |126         |
|JungleMaster  |Bronze  |Warwick      |Jungle   |2.3        |44      |40          |
|RogueAssassin |Diamond |Kha'Zix      |Jungle   |4.0        |59      |140         |
|RageFury      |Diamond |Draven       |ADC      |3.9        |60      |140   

In [18]:
spark.sql("""
select main_role, count(*) as number_of_players_per_role
from lakehouse.lol_statistics
group by main_role
order by main_role asc
""").show(truncate=False)

+---------+--------------------------+
|main_role|number_of_players_per_role|
+---------+--------------------------+
|ADC      |17                        |
|Jungle   |16                        |
|Mid      |27                        |
|Support  |26                        |
|Top      |23                        |
+---------+--------------------------+



In [19]:
df = spark.table("lakehouse.lol_statistics")

In [28]:
df.filter("rank='Diamond'").select("main_role","win_rate").show()

+---------+--------+
|main_role|win_rate|
+---------+--------+
|   Jungle|      59|
|      Mid|      58|
|   Jungle|      60|
|   Jungle|      60|
|  Support|      58|
|      Top|      58|
|      ADC|      57|
|   Jungle|      59|
|      Top|      57|
|      Mid|      58|
|      ADC|      60|
|  Support|      59|
|  Support|      58|
|   Jungle|      59|
|      ADC|      57|
|      ADC|      58|
|   Jungle|      60|
|      Mid|      59|
|      Mid|      57|
|   Jungle|      59|
+---------+--------+
only showing top 20 rows

